# Expected Average

2021-07-03

What we are interested in is determining a way to rank cards in our hand. How do we decide which cards to discard before the starter or cut card is turned? We can use the concept of the expected value or expected average of a hand and discard values.

According to [Wikipedia][link1], the expected value is generalization of a weighted average. We are interested in the expected average value of a typical hand discard. If we are dealt 6 cards and are required to discard 2 cards to the crib before the starter is turned. What is the expected average value of the hand we decide to keep given that we know the 6 cards we were dealt and do not exist in the deck any more.

[link1]: https://en.wikipedia.org/wiki/Expected_value


## References

- https://en.wikipedia.org/wiki/Expected_value
- http://www.cribbageforum.com/AnalyzeDiscardsPart1.htm
- http://www.cribbageforum.com/YourCrib.htm


In [1]:
%%javascript
//Disable autoscroll in the output cells
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import random

from multiprocessing import Pool
from functools import partial

from cribbage.cards import (
    Card,
    make_deck,
    display_hand,
    score_hand,
    score_hand_breakdown,
    hand_combinations,        
)

from cribbage.analytics import (
    expected_average,
    discard_max_hand_value,
    expected_average_crib,
)

# Expected Average Hand Value

The calculation is not complicated. We remove the 6 cards we were dealt from the deck, leaving 46 cards. We take the 4 card hand and determine the value of that hand for every possible starter/cut card left in the deck. We sum the value of each hand and divide that by the number of potential starter cards in the deck. This will give us the expected average value for the hand.

Let's assume we were dealt the following hand: 2C 3H 4D 5D 5S JS

Let's analyze the following 4 card hand: 3H, 4H, 5C, 5D, what can we expect in terms of the cut card? That is, based on what could be turned up as the cut card what kind of points can be expected, on average? 

We need to remove the cards that we know about from the deck so that we have $52 - 6 = 46$

| Starter | Card Frequency | Hand Value | Total |
|:-------:|:--------------:|:----------:|:-----:|
|A        |4               |   10       |     40|  
|2        |3               |   12       |     36|  
|3        |3               |   20       |     60|  
|4        |3               |   16       |     48|  
|5        |2               |   17       |     34|  
|6        |4               |   14       |     56|  
|7        |4               |   12       |     48|  
|8        |4               |   10       |     40|  
|9        |4               |   8        |     32|  
|10       |4               |   12       |     48|  
|J        |3               |   12       |     36|  
|Q        |4               |   12       |     48|  
|K        |4               |   12       |     48|
|         |                |   *Total:* |    574|

for the 4 card hand, the average points would be: $\frac{574}{46} = 12.48$

Table columns:
- Starter - This column simply lists the potential cut card.
- Card Frequency - This is the number of cards in the deck that are left based on the 6 cards you are dealt. For example, if I was dealt 2 aces, that would mean there are only two aces left in the deck. 
- Hand Value - This is the value of the hand including the cut/starter card
- Total - This is the product of the card frequency and hand value. Basically it is a weighting factor.

You sum the value in the total column and divide by the number of cards left in the deck, $52 - 6 = 46$ (weighted average). 

In [3]:
cards = [Card(*c) for c in ('2C', '3H', '4D', '5D', '5S', 'JS')]

hand = [Card(*c) for c in ('3H', '4D', '5D', '5S')]
discard  = [Card(*c) for c in ('2C', 'JS')]

hand_average = expected_average(hand, discard)
hand_value = score_hand(hand, None)

print(f"Hand    = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")

print(f"Hand Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")

Hand    = ['3♥', '4♦', '5♦', '5♠']
Discard = ['2♣', 'J♠']
Hand Value = 8
Average Value = 12.478


In [4]:
# Create a deck
deck = make_deck()

# get 6 cards
cards = list(random.sample(deck, 6))

# extract the discard
discard = cards[-2:]

# exclude the discard from the hand
hand = cards[:-2]

hand_average = expected_average(hand, discard)
hand_value = score_hand(hand, None)

print(f"Dealt   = {display_hand(cards, cool=True)}")
print(f"Hand    = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")

print(f"Hand Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")

Dealt   = ['J♥', '3♠', '2♥', 'T♠', 'J♦', 'J♣']
Hand    = ['J♥', '3♠', '2♥', 'T♠']
Discard = ['J♦', 'J♣']
Hand Value = 4
Average Value = 6.848


We could analyze 4 cards and assume no knowledge of the other cards in the deck. We could also add more cards to the discard, cards that we know are not in the deck.

# Determine Best Discard - Maximize Hand Expected Average

Now that we can calculate the expected average for any particular combination of hand cards and discard cards. We can use that to find the best set of cards to keep (or discard depending on how you look at it). This method will only attempt to maximize the expected average of the cards you keep in your hand. This may not be the best strategy overall. That may depend on being the dealer or the pone (you may want to maximize the cards in your crib or minimize the value of the dealer's crib).


We'll iterate through all of the 4 card combinations and choose the one with the largest expected average.


In [5]:
cards = [Card(*'KH'), Card(*'7D'), Card(*'9D'), Card(*'AD'), Card(*'8C'), Card(*'JD')] 

for i, hand in enumerate(hand_combinations(cards, combination_length=4)):
    value = score_hand(list(hand), None)
    
    # use a set to figure out what cards were discarded
    discard = list(set(cards) - set(hand))

    average = expected_average(
        list(hand),
        discard,
    )
   
    print(f'Hand = {display_hand(sorted(hand), cool=True)}, value = {value}, average = {average:.3f}')

Hand = ['K♥', 'A♦', '7♦', '9♦'], value = 0, average = 2.457
Hand = ['K♥', '7♦', '9♦', '8♣'], value = 5, average = 6.804
Hand = ['K♥', '7♦', '9♦', 'J♦'], value = 0, average = 2.609
Hand = ['K♥', 'A♦', '7♦', '8♣'], value = 2, average = 3.891
Hand = ['K♥', 'A♦', '7♦', 'J♦'], value = 0, average = 2.717
Hand = ['K♥', '7♦', 'J♦', '8♣'], value = 2, average = 4.043
Hand = ['K♥', 'A♦', '9♦', '8♣'], value = 0, average = 1.717
Hand = ['K♥', 'A♦', '9♦', 'J♦'], value = 0, average = 2.804
Hand = ['K♥', '9♦', 'J♦', '8♣'], value = 0, average = 1.826
Hand = ['K♥', 'A♦', 'J♦', '8♣'], value = 0, average = 1.978
Hand = ['A♦', '7♦', '9♦', '8♣'], value = 5, average = 7.891
Hand = ['A♦', '7♦', '9♦', 'J♦'], value = 4, average = 6.065
Hand = ['7♦', '9♦', 'J♦', '8♣'], value = 5, average = 7.783
Hand = ['A♦', '7♦', 'J♦', '8♣'], value = 2, average = 4.870
Hand = ['A♦', '9♦', 'J♦', '8♣'], value = 0, average = 2.696


In [6]:
hand = [Card(*'7D'), Card(*'9D'), Card(*'8C'), Card(*'JD')]
discard = [Card(*'KH'), Card(*'AD')]

hand_average = expected_average(hand, discard)
hand_value = score_hand(hand, None)

print(f"Dealt   = {display_hand(cards, cool=True)}")
print(f"Hand    = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")

print(f"Hand Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")

Dealt   = ['K♥', '7♦', '9♦', 'A♦', '8♣', 'J♦']
Hand    = ['7♦', '9♦', '8♣', 'J♦']
Discard = ['K♥', 'A♦']
Hand Value = 5
Average Value = 7.783


A random deck

In [7]:
# Create a deck
deck = make_deck()

# get 6 cards
cards = list(random.sample(deck, 6))

for i, hand in enumerate(hand_combinations(cards, combination_length=4)):
    value = score_hand(list(hand), None)
    
    # use a set to figure out what cards were discarded
    discard = list(set(cards) - set(hand))

    average = expected_average(
        list(hand),
        discard,
    )
   
    print(f'Hand = {display_hand(sorted(hand), cool=True)}, value = {value}, average = {average:.3f}')

Hand = ['K♦', '8♠', '5♣', '7♣'], value = 4, average = 6.478
Hand = ['K♦', 'A♣', '5♣', '7♣'], value = 2, average = 5.348
Hand = ['K♦', 'J♠', '5♣', '7♣'], value = 4, average = 6.457
Hand = ['8♠', 'A♣', '5♣', '7♣'], value = 2, average = 6.000
Hand = ['8♠', 'J♠', '5♣', '7♣'], value = 4, average = 6.717
Hand = ['J♠', 'A♣', '5♣', '7♣'], value = 2, average = 5.587
Hand = ['K♦', '8♠', 'A♣', '7♣'], value = 2, average = 3.913
Hand = ['K♦', '8♠', 'J♠', '7♣'], value = 2, average = 4.065
Hand = ['K♦', 'J♠', 'A♣', '7♣'], value = 0, average = 1.891
Hand = ['8♠', 'J♠', 'A♣', '7♣'], value = 2, average = 4.152
Hand = ['K♦', '8♠', 'A♣', '5♣'], value = 2, average = 4.217
Hand = ['K♦', '8♠', 'J♠', '5♣'], value = 4, average = 6.196
Hand = ['K♦', 'J♠', 'A♣', '5♣'], value = 4, average = 6.413
Hand = ['8♠', 'J♠', 'A♣', '5♣'], value = 2, average = 4.457
Hand = ['K♦', '8♠', 'J♠', 'A♣'], value = 0, average = 1.935


In [8]:
cards = [Card(*'7D'), Card(*'9D'), Card(*'8C'), Card(*'JD'), Card(*'KH'), Card(*'AD')]

result = discard_max_hand_value(cards)

hand = result['best_hand']
discard = result['best_discard']
hand_average = result['best_average']
hand_value = score_hand(hand, None)

print()
print(f"Hand = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")
print(f"Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")
print()

for row in result['messages']:
    print(row)



Hand = ['A♦', '7♦', '9♦', '8♣']
Discard = ['K♥', 'J♦']
Value = 5
Average Value = 7.891

 0 Hand = ['7♦', '9♦', 'J♦', '8♣'], value = 5, average = 7.783
 1 Hand = ['K♥', '7♦', '9♦', '8♣'], value = 5, average = 6.804
 2 Hand = ['A♦', '7♦', '9♦', '8♣'], value = 5, average = 7.891
 3 Hand = ['K♥', '7♦', '9♦', 'J♦'], value = 0, average = 2.609
 4 Hand = ['A♦', '7♦', '9♦', 'J♦'], value = 4, average = 6.065
 5 Hand = ['K♥', 'A♦', '7♦', '9♦'], value = 0, average = 2.457
 6 Hand = ['K♥', '7♦', 'J♦', '8♣'], value = 2, average = 4.043
 7 Hand = ['A♦', '7♦', 'J♦', '8♣'], value = 2, average = 4.870
 8 Hand = ['K♥', 'A♦', '7♦', '8♣'], value = 2, average = 3.891
 9 Hand = ['K♥', 'A♦', '7♦', 'J♦'], value = 0, average = 2.717
10 Hand = ['K♥', '9♦', 'J♦', '8♣'], value = 0, average = 1.826
11 Hand = ['A♦', '9♦', 'J♦', '8♣'], value = 0, average = 2.696
12 Hand = ['K♥', 'A♦', '9♦', '8♣'], value = 0, average = 1.717
13 Hand = ['K♥', 'A♦', '9♦', 'J♦'], value = 0, average = 2.804
14 Hand = ['K♥', 'A♦', 'J♦', 

In [9]:
# Create a deck
deck = make_deck()

# get 6 cards
cards = list(random.sample(deck, 6))


result = discard_max_hand_value(cards)

hand = result['best_hand']
discard = result['best_discard']
hand_average = result['best_average']
hand_value = score_hand(hand, None)

print()
print(f"Hand = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")
print(f"Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")
print()

for row in result['messages']:
    print(row)



Hand = ['9♥', '2♦', '9♦', '4♠']
Discard = ['Q♥', '6♣']
Value = 6
Average Value = 7.565

 0 Hand = ['9♥', '2♦', '9♦', '6♣'], value = 6, average = 7.217
 1 Hand = ['9♥', '2♦', '9♦', '4♠'], value = 6, average = 7.565
 2 Hand = ['9♥', 'Q♥', '2♦', '9♦'], value = 2, average = 3.304
 3 Hand = ['2♦', '9♦', '4♠', '6♣'], value = 4, average = 6.087
 4 Hand = ['Q♥', '2♦', '9♦', '6♣'], value = 2, average = 3.348
 5 Hand = ['Q♥', '2♦', '9♦', '4♠'], value = 2, average = 3.739
 6 Hand = ['9♥', '2♦', '4♠', '6♣'], value = 4, average = 6.087
 7 Hand = ['9♥', 'Q♥', '2♦', '6♣'], value = 2, average = 3.348
 8 Hand = ['9♥', 'Q♥', '2♦', '4♠'], value = 2, average = 3.739
 9 Hand = ['Q♥', '2♦', '4♠', '6♣'], value = 0, average = 2.261
10 Hand = ['9♥', '9♦', '4♠', '6♣'], value = 6, average = 7.478
11 Hand = ['9♥', 'Q♥', '9♦', '6♣'], value = 6, average = 6.957
12 Hand = ['9♥', 'Q♥', '9♦', '4♠'], value = 2, average = 3.304
13 Hand = ['Q♥', '9♦', '4♠', '6♣'], value = 2, average = 3.609
14 Hand = ['9♥', 'Q♥', '4♠', 

# NOTE - Expected Average Larger

We calculate the average of each hand and score that hand with the particular cut card. The typical analysis done on the internet doesn't look at the numbers in that detail. The following shows the differences. If you look at the table from the begining of the section, it is assumed that if you have 4 Aces, for example, they will produce hands of the same value.

| Starter | Card Frequency | Hand Value | Total |
|:-------:|:--------------:|:----------:|:-----:|
|A        |4               |   10       |     40|  
|2        |3               |   12       |     36|  
|3        |3               |   20       |     60|  
|4        |3               |   16       |     48|  
|5        |2               |   17       |     34|  
|6        |4               |   14       |     56|  
|7        |4               |   12       |     48|  
|8        |4               |   10       |     40|  
|9        |4               |   8        |     32|  
|10       |4               |   12       |     48|  
|J        |3               |   12       |     36|  
|Q        |4               |   12       |     48|  
|K        |4               |   12       |     48|
|         |                |   *Total:* |    574|

Under this assumption, the following set of cards would yield an average of 7.1957, below:

If we calcualte the actual value of the hand, including the suits, we get the following results (A higher expected average):

In [10]:
hand = [Card(*'7D'), Card(*'9D'), Card(*'8C'), Card(*'JD')]
discard = [Card(*'KH'), Card(*'AD')]

cards = hand + discard

deck = [c for c in make_deck() if c not in cards]

total = 0
for i, cut in enumerate(deck):
    hand_value = score_hand(hand, cut)    
    total += hand_value
    print(f"Cut {cut}; Hand Value = {hand_value}")
    
print(total/len(deck))

Cut AH; Hand Value = 5
Cut AC; Hand Value = 5
Cut AS; Hand Value = 5
Cut 2D; Hand Value = 10
Cut 2H; Hand Value = 5
Cut 2C; Hand Value = 5
Cut 2S; Hand Value = 5
Cut 3D; Hand Value = 10
Cut 3H; Hand Value = 5
Cut 3C; Hand Value = 5
Cut 3S; Hand Value = 5
Cut 4D; Hand Value = 10
Cut 4H; Hand Value = 5
Cut 4C; Hand Value = 5
Cut 4S; Hand Value = 5
Cut 5D; Hand Value = 12
Cut 5H; Hand Value = 7
Cut 5C; Hand Value = 7
Cut 5S; Hand Value = 7
Cut 6D; Hand Value = 13
Cut 6H; Hand Value = 8
Cut 6C; Hand Value = 8
Cut 6S; Hand Value = 8
Cut 7H; Hand Value = 12
Cut 7C; Hand Value = 12
Cut 7S; Hand Value = 12
Cut 8D; Hand Value = 17
Cut 8H; Hand Value = 12
Cut 8S; Hand Value = 12
Cut 9H; Hand Value = 10
Cut 9C; Hand Value = 10
Cut 9S; Hand Value = 10
Cut TD; Hand Value = 10
Cut TH; Hand Value = 5
Cut TC; Hand Value = 5
Cut TS; Hand Value = 5
Cut JH; Hand Value = 7
Cut JC; Hand Value = 7
Cut JS; Hand Value = 7
Cut QD; Hand Value = 10
Cut QH; Hand Value = 5
Cut QC; Hand Value = 5
Cut QS; Hand Value

In [11]:
hand = [Card(*'7D'), Card(*'9D'), Card(*'8C'), Card(*'JD')]
discard = [Card(*'KH'), Card(*'AD')]

hand_average = expected_average(hand, discard)
hand_value = score_hand(hand, None)

print(f"Dealt   = {display_hand(cards, cool=True)}")
print(f"Hand    = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")

print(f"Hand Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")

Dealt   = ['7♦', '9♦', '8♣', 'J♦', 'K♥', 'A♦']
Hand    = ['7♦', '9♦', '8♣', 'J♦']
Discard = ['K♥', 'A♦']
Hand Value = 5
Average Value = 7.783


I believe this was a simplifying assumption to make the analysis easier. 

>NOTE: My code and the basic hand analysis agree in terms of expected average values.

## Section Summary

At this point, we can calculate the optimal cards to discard from your hand. But how does that effect the crib. We'll explore that in the next section

# Crib - Expected Average Value

We can determine the expected average for any particular crib hand, can we do the same thing for the crib value? Yes, we can. This will lead us to a discard strategy depending whether we are the dealer or pone. Looking at the internet there are some resources. There are also many books. Here are a few:

- https://cliambrown.com/cribbage/methodology.php
- http://www.cribbageforum.com/YourCrib.htm

Unfortunately, they only provide simplified tables and nothing too in depth to be able to perform the calculations from first principles. It seems they may have been calculated on ancient machines. We'll follow the same approach we took in determining the expected average value for the hand.

## Method

What we'll do is:

1. Take the discard and pair them with all two card combinations remaining in the deck
1. Take a starter from the deck
1. Score these crib along with the starter and determine its average value
1. Search for the maximum value and the minimum value

Given a 6 card hand and 2 cards to discard to the crib. What is the average crib value?

We will calculate this by 

1. Creating a deck of cards and removing the 6 cards from the initial hand
2. Iterate through all 2 card combinations left within the deck
3. Combine the 2 cards with the discarded cards to form the crib
4. calculated the expected average of the crib combination - use the hand of cards that we keep as the discard to the method expected_average
5. accumulate the average crib values and the total number of cribs considered
6. divide the total by the number of crib hands to determine the crib average

In [13]:
hand = [Card(*'7D'), Card(*'9D'), Card(*'8C'), Card(*'JD')]
discard = [Card(*'KH'), Card(*'AD')]

cards = hand + discard

# remove the cards from the deck, this one will be for determing the two cards to finish the crib
deck = [c for c in make_deck() if c not in cards]


# max_value = 0
# max_crib = None

crib_averages = []
# iterate through every two card combination left in the deck so we can form a crib
for i, right in enumerate(hand_combinations(deck, combination_length=2), start = 1):
    crib = discard + list(right)
    
    hand_average = expected_average(crib, hand) # use the hand as the discard i.e. we know about those values    
#     print(f'{i:>3} Crib = {display_hand(crib, cool=True)} = {hand_average:.3f}')    
    
    crib_averages.append(hand_average)

crib_average = sum(crib_averages)/len(crib_averages)

print('---------------')
print(f"Dealt   = {display_hand(cards, cool=True)}")
print(f"Hand    = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")

hand_average = expected_average(hand, discard)
hand_value = score_hand(hand, None)

print(f"Hand Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")

print(f'Crib Average = {crib_average:.3f}')

---------------
Dealt   = ['7♦', '9♦', '8♣', 'J♦', 'K♥', 'A♦']
Hand    = ['7♦', '9♦', '8♣', 'J♦']
Discard = ['K♥', 'A♦']
Hand Value = 5
Average Value = 7.783
Crib Average = 4.005


In [14]:
hand = [Card(*'7D'), Card(*'9D'), Card(*'8C'), Card(*'JD')]
discard = [Card(*'KH'), Card(*'AD')]

crib_average = expected_average_crib(hand, discard)

print('---------------')
print(f"Dealt   = {display_hand(hand + discard, cool=True)}")
print(f"Hand    = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")

hand_average = expected_average(hand, discard)
hand_value = score_hand(hand, None)

print(f"Hand Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")

print(f'Crib Average = {crib_average:.3f}')

---------------
Dealt   = ['7♦', '9♦', '8♣', 'J♦', 'K♥', 'A♦']
Hand    = ['7♦', '9♦', '8♣', 'J♦']
Discard = ['K♥', 'A♦']
Hand Value = 5
Average Value = 7.783
Crib Average = 4.005


## Multiprocessing

In [15]:
def _calc_crib_average(pair, discard=None, base_cards=None):
    """
    Form a crib hand from the two base_cards and the cards in the pair. 
    Provide discard cards to get a better estimate of the average.
    
    Returns the result
    
    This is for multiprocessing.
    
    """
    
    assert len(base_cards) == 2
    assert len(pair) == 2
    
    crib = base_cards + list(pair)
    
    # use the hand as the discard i.e. we know about those values    
    hand_average = expected_average(crib, discard) 
    
    return hand_average

In [16]:
hand = [Card(*'7D'), Card(*'9D'), Card(*'8C'), Card(*'JD')]
discard = [Card(*'KH'), Card(*'AD')]

cards = hand + discard

# remove the cards from the deck, this one will be for determing the two cards to finish the crib
deck = [c for c in make_deck() if c not in hand + discard]

fn = partial(_calc_crib_average, discard=hand, base_cards=discard)

# Use max cores
with Pool(processes=None) as p:
    crib_averages = p.map(fn, hand_combinations(deck, combination_length=2))

crib_average = sum(crib_averages)/len(crib_averages)

print('---------------')
print(f"Dealt   = {display_hand(cards, cool=True)}")
print(f"Hand    = {display_hand(hand, cool=True)}")
print(f"Discard = {display_hand(discard, cool=True)}")

hand_average = expected_average(hand, discard)
hand_value = score_hand(hand, None)

print(f"Hand Value = {hand_value}")
print(f"Average Value = {hand_average:.3f}")

print(f'Crib Average = {crib_average:.3f}')

---------------
Dealt   = ['7♦', '9♦', '8♣', 'J♦', 'K♥', 'A♦']
Hand    = ['7♦', '9♦', '8♣', 'J♦']
Discard = ['K♥', 'A♦']
Hand Value = 5
Average Value = 7.783
Crib Average = 4.005


=====================
Old stuff below

These numbers agree with what I have in the books and tables on the internet. I think these values will be more accurate because it does take into account flushes, nobs and other things that I think the other approaches don't accommodate for. 

# Analyze Average Discard Value

- https://cliambrown.com/cribbage/methodology.php
- http://www.cribbageforum.com/YourCrib.htm

There are only discard tables available. I think I need to follow the same approach as the previous section.

1. Take the cards to discard and pair them with two other cards from the remaining cards
1. Take a starter from the deck
1. score these crib along with the starter and determine its average value
1. Search for the maximum value and the minimum value

In [ ]:
def average_crib_value(deck, discard, **kwargs):
    """
    The player is dealt 6 cards which are removed from 
    the deck. They have picked 2 cards to discard to the crib.    
    This method calculates the average crib value, 
    given the remaining cards in the deck (46) and 2 discard cards
    intended for the crib.
    
    Essentially, it uses the remaining cards in the deck, each pair in turn, 
    as the potential crib mates and an extra card for the starter. 
    
    Parameters
    ----------
    deck - iterable - a list of cards that are remaining in the deck. There should be 46.
    discard - Hand object - a list of cards that will be part of the crib. There should be 2 cards.
    
    Returns
    -------
    
    The average crib value.
    
    """
   
#     assert len(deck) == 46
    assert len(discard) == 2
    
    verbose = False if 'verbose' not in kwargs else kwargs['verbose']
    
    total = 0.0 
    count = 0
        
    for i, combo in enumerate(combinations(deck, 3), 1):
        c1, c2, cut = combo
        crib = Hand([c1, c2])
        crib.extend(discard)
        
        scores, counts = score_hand(crib, cut, is_crib=True)    
        value = sum(scores.values())
        total += value
        count += 1
        
        if verbose:
            print('{:>2} Crib = {} Cut = {} Points = {}'.format(i, crib.sorted(), cut, value))
        
    average = total/count
        
    return average

In [ ]:
hand = Hand([Card('4', 'C'),  Card('Q', 'C'), Card('A','C'), Card('A', 'D')])
discard = Hand([Card('5', 'D'), Card('Q', 'H')])

deck = make_deck()

for c in hand:
    deck.remove(c)

for c in discard:
    deck.remove(c)
    
print('Candidate Hand = {} '.format(hand.sorted()))
print('Discard: ', discard.sorted())
print('-------')

# find the average hand value
average = average_crib_value(deck, discard, verbose=False)

print()
print('Average crib value = {:.4f}'.format(average))        
print('-------')
print()

## Construct Crib Discard Table

On my computer it takes about 6 seconds to compute the average value of the discard to the crib. We'll create a dictionary of all the possible pairings of cards for the discard and add them to a lookup dictionary. Once the dictionary has been calculated we'll store it as .json so we don't need to do it every time.

I don't think counting flushes will make much of a difference to this process. To save computation time, I'll reduce all duplicate ranks from the deck


In [ ]:
# testing methods to reduce the number of cards that we need to consider in the deck
deck = make_deck()

# extract only a single suite
deck = sorted(deck)[0::2]
print(deck)
print(len(deck))

In [ ]:
deck = make_deck()
candidates = sorted(deck)[0::2]

average_crib_values = {}

# attempt to load the pre-calculated averages
try:
    
    with open('crib_discard.json', 'r') as fp:
        average_crib_values = json.load(fp)
    
except FileNotFoundError:
    # there is no pre-calculated dictionary
    pass
    
for i, combo in enumerate(combinations(candidates, 2), 1):
        discard = Hand(combo)
        key = str([c.rank for c in discard])
        
        if key not in average_crib_values:        
            reduced_deck = list(set(deck) - set(discard))
            average = average_crib_value(reduced_deck, discard, verbose=False)
            average_crib_values[key] = average
            print('{:<2} Discard = {} Average = {:.4f}'.format(i, key, average_crib_values[key]))            

In [ ]:
# dump the averages into a json file so we don't have to recalculate it every time
with open('crib_discard.json', 'w') as fp:
    json.dump(average_crib_values, fp, indent=4)

In [ ]:
print('Overall average crib points = {:.4f}'.format(sum(average_crib_values.values())/len(average_crib_values)))

The values look good and are close to a few of the tables that I have laying around. Let us bring them all together in the next section.

In [ ]:
# testing a way to make the dictionary key out of a list
m = Hand([Card('4H'), Card('6H')])
print(m)
print(m.cool_display())

key = str(m)
print(key)
print(type(key))


# Expected Average

For the dealer the expected average is the Average Hand Value + the Average Discard Value

For the pone the expected average is the Average Hand Value - the Average Discard Value


In [ ]:
hand = Hand([Card('4', 'C'),  Card('Q', 'C'), Card('A','C'), Card('A', 'D'), Card('5', 'D'), Card('Q', 'H')])
deck = make_deck()

for c in hand:
    deck.remove(c)
    

## As Dealer

As the dealer, we want to maximize the amount of points we have in our hand and in the crib. I think the best approach is to maximize the value of the hand as the crib, on average is only valued at about 4.88 points.

In [ ]:
print('Candidate Hand = {} '.format(hand.sorted()))
print('')

score, best_hand = determine_best_hand(hand,verbose=False)
print('---------')
print('Best Hand      = {}'.format(best_hand))
print('Hand Value     = {}'.format(score))

# find the average hand value
hand_average = average_hand_value(deck, best_hand)
print('Average hand value = {:.4f}'.format(hand_average))  

discard = Hand(list(set(hand) - set(best_hand)))
print('Discard: ', discard.sorted())
print('-------')

# find the average crib value. This takes into account flushes and all suits. 
# No simplification is made, the calculations are correct as far as I can tell.
average_crib = average_crib_value(deck, discard, verbose=False)

# in the cached dictionary of pre-calculated values suits where basically ignored 
# so the values will be slightly different then the other method
# key = str([c.rank for c in discard])
key = str(sorted([c.rank for c in discard]))
average_crib_precalc = average_crib_values[key]

print()
print('Average crib value                  = {:.4f}'.format(average_crib))        
print('Average crib value (pre-calculated) = {:.4f}'.format(average_crib_precalc))        
print('-------')
print()

print('Expected Average - Dealer = {}'.format(hand_average + average_crib))
print('Expected Average - Dealer = {}'.format(hand_average + average_crib_precalc))

## As Pone

The strategy for the pone is different. They want to maximize the average value of their hand while at the same time minimize the average value of the crib. This will take a little different strategy.


In [ ]:
print('Candidate Hand = {} '.format(hand.sorted()))
print('')

score, best_hand = determine_best_hand(hand,verbose=False)
print('---------')
print('Best Hand      = {}'.format(best_hand))
print('Hand Value     = {}'.format(score))

# find the average hand value
hand_average = average_hand_value(deck, best_hand)
print('Average hand value = {:.4f}'.format(hand_average))  

discard = Hand(list(set(hand) - set(best_hand)))
print('Discard: ', discard.sorted())
print('-------')

# find the average crib value. This takes into account flushes and all suits. 
# No simplification is made, the calculations are correct as far as I can tell.
average_crib = average_crib_value(deck, discard, verbose=False)

# in the cached dictionary of pre-calculated values suits where basically ignored 
# so the values will be slightly different then the other method
# key = str([c.rank for c in discard])
key = str(sorted([c.rank for c in discard]))
average_crib_precalc = average_crib_values[key]

print()
print('Average crib value                  = {:.4f}'.format(average_crib))        
print('Average crib value (pre-calculated) = {:.4f}'.format(average_crib_precalc))        
print('-------')
print()

print('Expected Average - Pone = {}'.format(hand_average - average_crib))
print('Expected Average - Pone = {}'.format(hand_average - average_crib_precalc))

Maximizing the value of the pone's hand doesn't yield an optimal strategy for the pone overall. In the above case it looks as though the pone can only expect about 1.5 points on average even though the hand value is 8.4! We need a different strategy to minimize the loss in points.

----

In [ ]:
def four_card_hand_scores(hand, **kwargs):
    """
    Takes a hand of more than 4 cards and returns a list containing 
    every 4 card hand combination along with the point value, point value dictionary and
    a counts dictionary outlining the number of matches for the point value dictionary.
            
    """
    hands = []
    
    for combo in hand.every_combination(count=4):
        new_hand = Hand(combo).sorted()        
        
        # we are only dealing with 4 cards, ignoring a cut card
        scores, counts = score_hand(new_hand, None) 
        score = sum(scores.values())
        
        hands.append((new_hand, score, scores, counts))
    
    return hands

In [ ]:
def average_hand_values(deck, hands, **kwargs):
    """
    Calculates the average value for the hands.
    
    Parameters
    ----------
    deck - the cards left unturned as far as the person (pone or dealer) 
           that are used to calculate the average hand value. 
    
    hands - tuple(Hand, number, dictionary, dictionary) - the list of hands to evaluate 
            the average values for.
            
    Returns
    -------
    a list of tuples similar to the input list of tuples that contain the hand,
    the value of the hand and the average value.
    
    """
    
    assert len(deck) == 46

    average_hands = []
    for hand_tuple in hands:
        hand, hand_value, *_ = hand_tuple
        average_value = average_hand_value(deck, hand)
    
        average_hands.append((hand, hand_value, average_value))
    
    return average_hands

In [ ]:
def average_discard_values(hand, average_hands, **kwargs):
    
    average_discards = []
    for i, a in enumerate(average_hands):
        average_hand, value, average = a
        discard = list(set(hand) - set(average_hand))
    #     average_crib = average_crib_value(deck, discard, verbose=False)

        try:

            key = str(sorted([c.rank for c in discard]))
            average_crib_precalc = average_crib_values[key]

        except KeyError:        
            key = str(sorted([c.rank for c in discard], reverse=True))
            average_crib_precalc = average_crib_values[key]

        average_discards.append((average_hand, 
                                 value, 
                                 average, 
                                 discard,
                                 average_crib_precalc))
        
    return average_discards

In [ ]:
hands = sorted(four_card_hand_scores(hand), key=lambda x:x[1], reverse=True)
average_hands = sorted(average_hand_values(deck, hands), key=lambda x:x[2], reverse=True)
average_discards = sorted(average_discard_values(hand, average_hands), key=lambda x:x[4])
   
row = '{:<3} {} = {} -> Average = {:.4f} - Discard = {} -> Average = {:.4f} -> EA = {:.4f}'
for i, ad in enumerate(average_discards):    
    print(row.format(i,*ad, ad[2] - ad[4]))

By minimizing the average points discarded to the crib, we maximize our expected average points for the hand.